In [1]:
from hyperopt import fmin, hp, tpe, SparkTrials, STATUS_OK
from hyperopt.early_stop import no_progress_loss

# from hpsklearn import any_regressor, any_preprocessing
# import sklearn.preprocessing
from sklearn.linear_model import LinearRegression

from lightgbm import LGBMRegressor
from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import Ridge

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR


In [2]:
lag_options = [ [1], [1, 2], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 25], [1, 2, 25, 26], [1, 2, 3, 25, 26, 27], [1, 2, 3, 4, 25, 26, 27, 28], [1, 2, 3, 4, 5, 25, 26, 27, 28, 29], [1, 2, 3, 4, 5, 6, 25, 26, 27, 28, 29, 30], [1, 2, 3, 4, 5, 6, 7, 25, 26, 27, 28, 29, 30, 31], [1, 2, 3, 4, 5, 6, 7, 8, 25, 26, 27, 28, 29, 30, 31, 32], [1, 2, 3, 4, 5, 6, 7, 8, 9, 25, 26, 27, 28, 29, 30, 31, 32, 33], [1, 25, 49], [1, 2, 25, 26, 49, 50], [1, 2, 3, 25, 26, 27, 49, 50, 51], [1, 2, 3, 4, 25, 26, 27, 28, 49, 50, 51, 52], [1, 2, 3, 4, 5, 25, 26, 27, 28, 29, 49, 50, 51, 52, 53], [1, 2, 3, 4, 5, 6, 25, 26, 27, 28, 29, 30, 49, 50, 51, 52, 53, 54], [1, 2, 3, 4, 5, 6, 7, 25, 26, 27, 28, 29, 30, 31, 49, 50, 51, 52, 53, 54, 55], [1, 2, 3, 4, 5, 6, 7, 8, 25, 26, 27, 28, 29, 30, 31, 32, 49, 50, 51, 52, 53, 54, 55, 56], [1, 2, 3, 4, 5, 6, 7, 8, 9, 25, 26, 27, 28, 29, 30, 31, 32, 33, 49, 50, 51, 52, 53, 54, 55, 56, 57]]

In [3]:
linreg = LinearRegression()
lgbm = LGBMRegressor()
# xgb = XGBRegressor()
catboost = CatBoostRegressor()
sgd = SGDRegressor()
kernelridge = KernelRidge()
elastic = ElasticNet()
bayesianridge = BayesianRidge()
# gradientboosting = GradientBoostingRegressor(n_estimators=100)
svr = SVR()
ridge = Ridge()

# all_regressors = [linreg, lgbm, xgb, catboost, sgd, kernelridge, elastic, bayesianridge, gradientboosting, svr]
all_regressors = [linreg, lgbm, catboost, sgd, kernelridge, elastic, bayesianridge, svr]

def get_params():
    search_space = {#"price_1": hp.uniform("price_1", -1, 1),
                    # "cap_learning_data": hp.quniform("cap_learning_data", 300, 800, 100),
                    # "update_learner_interval": hp.quniform("update_learner_interval", 1, 4, 1),
                    "cap_learning_data": hp.choice("cap_learning_data", [500]),
                    "update_learner_interval": hp.choice("update_learner_interval", [4]),
                    "load_lags": hp.choice("load_lags", lag_options),
                    "solar_lags": hp.choice("solar_lags", lag_options),
                    # "load_regressor": hp.choice("load_regressor", [ridge]),
                    # "solar_regressor": hp.choice("solar_regressor", [ridge]),

                    # "load_regressor": any_regressor("my_reg1"),
                    # "solar_regressor": any_regressor("my_reg2"),
                    # "load_transformer": any_preprocessing("my_pre1"),
                    # "solar_transformer": any_preprocessing("my_pre2")
                    }
    return search_space

In [ ]:
from traineval.evaluation.tune_evaluation import evaluate

best_params = fmin(
    fn=evaluate,
    space=get_params(),
    algo=tpe.suggest,  # NOTE: You cannot use atpe.suggest with SparkTrials, then use tpe.suggest
    max_evals=1000,
    trials=SparkTrials(18),
    # early_stop_fn=no_progress_loss(400)
)
best_params

  0%|          | 0/1000 [00:00<?, ?trial/s, best loss=?]

hyperopt.tpe INFO  build_posterior_wrapper took 0.000000 seconds
hyperopt.tpe INFO  TPE using 0 trials
hyperopt.tpe INFO  build_posterior_wrapper took 0.078125 seconds
hyperopt.tpe INFO  TPE using 1/1 trials with best loss inf
hyperopt.tpe INFO  build_posterior_wrapper took 0.015630 seconds
hyperopt.tpe INFO  TPE using 2/2 trials with best loss inf
hyperopt.tpe INFO  build_posterior_wrapper took 0.000000 seconds
hyperopt.tpe INFO  TPE using 3/3 trials with best loss inf
hyperopt.tpe INFO  build_posterior_wrapper took 0.000000 seconds
hyperopt.tpe INFO  TPE using 4/4 trials with best loss inf
hyperopt.tpe INFO  build_posterior_wrapper took 0.000000 seconds
hyperopt.tpe INFO  TPE using 5/5 trials with best loss inf
hyperopt.tpe INFO  build_posterior_wrapper took 0.000000 seconds
hyperopt.tpe INFO  TPE using 6/6 trials with best loss inf
hyperopt.tpe INFO  build_posterior_wrapper took 0.078140 seconds
hyperopt.tpe INFO  TPE using 7/7 trials with best loss inf
hyperopt.tpe INFO  build_post

  0%|          | 1/1000 [06:29<108:09:07, 389.74s/trial, best loss: 0.9226396885252065]

hyperopt.tpe INFO  build_posterior_wrapper took 0.000000 seconds
hyperopt.tpe INFO  TPE using 19/19 trials with best loss 0.922640
py4j.clientserver INFO  Closing down clientserver connection
py4j.clientserver INFO  Closing down clientserver connection


  0%|          | 3/1000 [06:31<28:11:28, 101.79s/trial, best loss: 0.9226396885252065] 

...

hyperopt.tpe INFO  build_posterior_wrapper took 0.015626 seconds
hyperopt.tpe INFO  TPE using 645/645 trials with best loss 0.920134
py4j.clientserver INFO  Closing down clientserver connection
py4j.clientserver INFO  Closing down clientserver connection


 63%|██████▎   | 629/1000 [3:55:22<17:43,  2.87s/trial, best loss: 0.9201335479845877]

hyperopt.tpe INFO  build_posterior_wrapper took 0.017998 seconds
hyperopt.tpe INFO  TPE using 646/646 trials with best loss 0.920134
hyperopt.tpe INFO  build_posterior_wrapper took 0.023412 seconds
hyperopt.tpe INFO  TPE using 647/647 trials with best loss 0.920134
py4j.clientserver INFO  Closing down clientserver connection


 63%|██████▎   | 630/1000 [3:55:44<44:23,  7.20s/trial, best loss: 0.9201335479845877]

hyperopt.tpe INFO  build_posterior_wrapper took 0.015642 seconds
hyperopt.tpe INFO  TPE using 648/648 trials with best loss 0.920134


In [ ]:
best_params